In [2]:
pip show google-ads

Name: google-ads
Version: 24.1.0
Summary: Client library for the Google Ads API
Home-page: https://github.com/googleads/google-ads-python
Author: Google LLC
Author-email: googleapis-packages@google.com
License: Apache 2.0
Location: C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages
Requires: google-api-core, google-auth-oauthlib, googleapis-common-protos, grpcio, grpcio-status, proto-plus, protobuf, PyYAML
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [27]:
import mysql.connector
from mysql.connector import Error
import schedule
import time
import requests
from google.ads.googleads.client import GoogleAdsClient
from facebook_business.api import FacebookAdsApi
from facebook_business.adobjects.adaccount import AdAccount

In [29]:
def create_server_connection(host_name, user_name, user_password, db_name):
    connection = None
    try:
        connection = mysql.connector.connect(
            host=host_name,
            user=user_name,
            password=user_password,
            database=db_name
        )
        print("MySQL Database connection successful")
    except Error as err:
        print(f"Error: '{err}'")
    return connection

host_name = "host_name"
user_name = "Mokereri"
user_password = "Nerd***"
db_name = "ADVERT_DB"

connection = create_server_connection(host_name, user_name, user_password, db_name)

MySQL Database connection successful


In [30]:
# Create the schema within ADVERT_DB
create_schema_query = "CREATE SCHEMA IF NOT EXISTS Public_schema;"
execute_query(connection, create_schema_query)

# Create tables within the Public_schema
create_facebook_ads_table = """
CREATE TABLE IF NOT EXISTS Public_schema.facebook_ads (
    ad_id VARCHAR(255) PRIMARY KEY,
    campaign_name VARCHAR(255),
    adset_name VARCHAR(255),
    ad_name VARCHAR(255),
    impressions INT,
    clicks INT,
    spend DECIMAL(10, 2),
    date DATE
);
"""

create_google_ads_table = """
CREATE TABLE IF NOT EXISTS Public_schema.google_ads (
    ad_id VARCHAR(255) PRIMARY KEY,
    campaign_name VARCHAR(255),
    adgroup_name VARCHAR(255),
    ad_name VARCHAR(255),
    impressions INT,
    clicks INT,
    spend DECIMAL(10, 2),
    date DATE
);
"""

create_orders_table = """
CREATE TABLE IF NOT EXISTS Public_schema.orders (
    order_id VARCHAR(255) PRIMARY KEY,
    order_date DATE,
    customer_id VARCHAR(255),
    product_id VARCHAR(255),
    quantity INT,
    total_amount DECIMAL(10, 2),
    source VARCHAR(255)
);
"""

# Execute the table creation queries within Public_schema
execute_query(connection, create_facebook_ads_table)
execute_query(connection, create_google_ads_table)
execute_query(connection, create_orders_table)

Query successful
Query successful
Query successful
Query successful


In [ ]:
# Extract data from Facebook Ads
def fetch_facebook_ads():
    access_token = 'your_facebook_access_token'
    ad_account_id = 'your_ad_account_id'
    FacebookAdsApi.init(access_token=access_token)
    account = AdAccount(f'act_{ad_account_id}')
    fields = ['ad_id', 'campaign_name', 'adset_name', 'ad_name', 'impressions', 'clicks', 'spend', 'date_start']
    params = {
        'time_range': {'since': '2023-01-01', 'until': '2024-12-31'},
        'level': 'ad'
    }
    ads = account.get_insights(fields=fields, params=params)
    for ad in ads:
        cursor.execute("""
            INSERT INTO facebook_ads (ad_id, campaign_name, adset_name, ad_name, impressions, clicks, spend, date)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (ad['ad_id'], ad['campaign_name'], ad['adset_name'], ad['ad_name'], ad['impressions'], ad['clicks'], ad['spend'], ad['date_start']))

# Extract data from Google Ads
def fetch_google_ads():
    client = GoogleAdsClient.load_from_storage('path/to/google-ads.yaml')
    query = """
        SELECT ad_group_ad.ad.id, campaign.name, ad_group.name, ad_group_ad.ad.name, metrics.impressions, metrics.clicks, metrics.cost_micros, segments.date
        FROM ad_group_ad
        WHERE segments.date DURING LAST_30_DAYS
    """
    response = client.service.google_ads.search(customer_id='your_google_ads_customer_id', query=query)
    for row in response:
        ad_id = row.ad_group_ad.ad.id
        campaign_name = row.campaign.name
        adgroup_name = row.ad_group.name
        ad_name = row.ad_group_ad.ad.name
        impressions = row.metrics.impressions
        clicks = row.metrics.clicks
        spend = row.metrics.cost_micros / 1e6  # Convert micros to currency
        date = row.segments.date
        cursor.execute("""
            INSERT INTO google_ads (ad_id, campaign_name, adgroup_name, ad_name, impressions, clicks, spend, date)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
        """, (ad_id, campaign_name, adgroup_name, ad_name, impressions, clicks, spend, date))

# Extract data from Orders
def fetch_orders():
    # If orders are already stored in the database, skip. If from API, similar process as above.
    pass

def update():
    connection = create_server_connection(host_name, user_name, user_password, db_name)
    cursor = connection.cursor()
    fetch_facebook_ads()
    fetch_google_ads()
    fetch_orders()
    connection.commit()
    connection.close()

host_name = "127.0.0.1"
user_name = "Mokereri"
user_password = "Kay@2030"
db_name = "ADVERT_DB"

schedule.every().day.at("00:00").do(update)

while True:
    schedule.run_pending()
    time.sleep(1)

In [26]:
# Create views
create_total_ad_spend_view = """
CREATE OR REPLACE VIEW total_ad_spend AS
SELECT
    SUM(spend) AS total_spend,
    'Facebook' AS platform
FROM facebook_ads
UNION ALL
SELECT
    SUM(spend) AS total_spend,
    'Google' AS platform
FROM google_ads;
"""

create_cost_per_order_view = """
CREATE OR REPLACE VIEW cost_per_order AS
SELECT
    o.source,
    SUM(o.total_amount) AS total_sales,
    COUNT(o.order_id) AS total_orders,
    SUM(a.spend) AS total_ad_spend,
    (SUM(a.spend) / COUNT(o.order_id)) AS cost_per_order
FROM orders o
JOIN (
    SELECT ad_id, spend, 'Facebook' AS source FROM facebook_ads
    UNION ALL
    SELECT ad_id, spend, 'Google' AS source FROM google_ads
) a ON o.source = a.source
GROUP BY o.source;
"""

# Commit the changes and close the connection
if connection.is_connected():
    connection.commit() 
    connection.close()
    print("MySQL connection is closed")


MySQL connection is closed
